# 교차 검증과 그리드 서치

## 검증 세트

In [ ]:
import pandas as pd

# 와인 데이터 받아오기(class 0 : 레드 와인, 1 : 화이트 와인)
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# class 열을 제외한 나머지 3개열을 입력 데이터로 사용
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# class 열을 target으로 사용
target = wine['class'].to_numpy()

In [ ]:
# train set과 test set으로 나누기 (train set: 80%, test set: 20%)
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
# train set을 다시 train set과 validation set으로 나누기 (train set: 80%, validation set: 20%)
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
# DecisionTreeClassifier 알고리즘 사용
from sklearn.tree import DecisionTreeClassifier

# 모델 생성
dt = DecisionTreeClassifier(random_state=42)
# 모델 훈련
dt.fit(sub_input, sub_target)

# 모델 평가
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target)) # test set이 아닌, validation set 이용

0.9971133028626413
0.864423076923077


## 교차 검증

In [ ]:
# k-fold cross validation 사용 (기본 k값은 5)
from sklearn.model_selection import cross_validate

# 객체 생성(모델 훈련) # (fit_time : 모델 훈련 시간, score_time : 모델 검증 시간, test_score : 검증 폴드 점수(이름만 test_score)) 세 키를 가진 딕셔너리 반환 # cross_val_score 사용 시 test_score 값만 반환
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00931716, 0.00749564, 0.00773239, 0.00731683, 0.00710797]), 'score_time': array([0.00109315, 0.00111032, 0.00101209, 0.00106931, 0.00115085]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

# 교차 검증의 최종 점수(검증 폴드 점수의 평균)
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# splitter로 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold

# 객체 생성(모델 훈련)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # StratifiedKFold에 아무것도 설정해주지 않았기 때문에 앞서 수행한 교차 검증과 동일한 결과
# 교차 검증의 최종 점수(검증 폴드 점수의 평균)
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # n_splits : k값(k-fold cross validation) 설정
# 객체 생성(모델 훈련)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
# 교차 검증의 최종 점수(검증 폴드 점수의 평균)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [ ]:
# GridSearch 사용 (기본 k값은 5)
from sklearn.model_selection import GridSearchCV

# 탐색할 하이퍼파라미터를 딕셔너리로 생성
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# 객체 생성(모델 생성)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs : 사용할 CPU 코어 수, -1일 경우 시스템에 있는 모든 CPU 코어 사용

In [ ]:
# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# 그리드 서치로 찾은 최적의 하이퍼파라미터로 전체 train set를 훈련시킨 모델 불러오기
dt = gs.best_estimator_
# 모델 평가(train set)
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# 그리드 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 각 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
# 그리드 서치로 찾은 최적의 하이퍼파라미터 # gs.best_params_와 동일
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
# 여러 하이퍼파라미터 조합
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
# 객체 생성(모델 생성)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# 그리드 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# 최적의 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [ ]:
# 주어진 범위에서 고르게 값을 뽑음(균등 분포에서 샘플링) # uniform은 실숫값 # randint는 정숫값
from scipy.stats import uniform, randint

In [ ]:
# 0에서 10 사이의 범위에서 10개의 정수 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 7, 6, 8, 9, 3, 8, 3, 1, 4])

In [ ]:
# 균등 분포인지 확인하기 위해 1000개를 샘플링하고 각 숫자의 개수 세기
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([116, 105,  95, 100,  84,  90,  97,  95, 107, 111]))

In [ ]:
# 0에서 1 사이의 범위에서 10개의 실수 샘플링
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.07156624, 0.51330724, 0.78244744, 0.14237963, 0.05055468,
       0.13124955, 0.15801332, 0.99110938, 0.08459786, 0.92447632])

In [ ]:
# 여러 하이퍼파라미터 조합(범위만 설정)
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
# RandomSearch 사용 (기본 k값은 5)
from sklearn.model_selection import RandomizedSearchCV

# 객체 생성(모델 생성)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter : 샘플링 횟수
# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
# 최적의 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터로 전체 train set를 훈련시킨 모델 불러오기
dt = gs.best_estimator_

# 모델 평가(test set) # 테스트 세트 점수는 검증 세트 점수보다 조금 작은 것이 일반적
print(dt.score(test_input, test_target))

0.86


## 확인문제
#### DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 훈련한 뒤 테스트 세트에서의 성능 확인하기
#### Tip! 기본 splitter는 'best'로 각 노드에서 최선의 분할을 찾음
#### Tip! splitter가 'random'이면 무작위로 분할한 다음 가장 좋은 것을 고름
#### - 테스트 세트에서의 성능이 내려갔음을 확인할 수 있음

In [ ]:
# 객체 생성(모델 생성)
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter : 샘플링 횟수
# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)
# 최적의 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(np.max(gs.cv_results_['mean_test_score']))

# 랜덤 서치로 찾은 최적의 하이퍼파라미터로 전체 train set를 훈련시킨 모델 불러오기
dt = gs.best_estimator_
# 모델 평가(test set)
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
